# Calibration the Sr/Ca paleothermometer: an example from Dry Tortugas

This notebook will assist you in the data exploration process. If you know Python, feel free to change as needed to answer the questions.

For Python novice, time to get your feet a little wet! Don't worry, nothing difficult but you may be asked to copy and paste cells and change the indices as needed. 

First let's import necessary packages for our work. To run the cell below, with click on play or `shift+enter`.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pymc3 as pm
from sklearn.linear_model import LinearRegression

## Obtaining the data.

For this calibration exercise, we will be using the Sr/Ca data from [DeLong et al. (2011)](https://www.sciencedirect.com/science/article/pii/S0031018211002501?casa_token=u1x_ZYnm_mIAAAAA:rU7n-8jHh2g5UPgGHs5h1iptBXVa6rfvKFxpOwgMjHgB6g4jUZ9oRppzJz7O5UQHDlc1U3xhYSg). 

The data is available through the database maintained by NOAA NCEI. The cell below will download the data into the workspace. Note that Colab doesn't keep data in folder. If you close this window, you may have to redownload it. 

In [ ]:
!wget https://www.ncei.noaa.gov/pub/data/paleo/coral/atlantic/tortugas2011.xls

Now that we have the data, let's import it into our workspace using [pandas](https://pandas.pydata.org). Pandas dataframes are one of the most useful data models in data science. Many other libraries for plotting and analysis natively use them. 

Pandas allow you to create a dataframe from a table in excel using the function `read_excel`. In the cell below, I am creating a dataframe from the file we have just downloaded, using the second sheet (this is the confusing part; Python indexing starts at 0 so the first sheet is actually 0), and considering that the headers for the columns are in lines 1,2,3 (so 0,1,2 in Python).

In [ ]:
df = pd.read_excel('tortugas2011.xls', sheet_name=1,header=[0,2])
df.head()

**Question 1:** Based on the ReadMe in the excel file and the manuscript, describe each column in the file. It doesn't have to be an extensive description. It's just to orient us. 

The next step is to explore the data and think about our calibration exercise.

## Data Exploration

The cell below is just a practical guide to the names of the columns and their respective indices. Execute it.

In [ ]:
headers = list(df.columns.values)
for idx,item in enumerate(headers):
  print(str(idx)+": "+str(item))

The format is as followed:
column index: variable name.

There are multiple ways to select columns in a pandas dataframe. You can either do so by name (by copying/pasting the headers above) or using `iloc` and the column number. I'll be using the second option since the column names are long.

Let's first plot the Sr/Ca record of the species of corals in the dataset. 

About the code:
* The first line allows you to change the size of the figure to your convenience.
* The second to fourth line plots the data. Here we have three Sr/Ca recods. The plot function takes (x,y,label). x is time (column 0), y is the Sr/Ca values (column 1,4,7), label is of your choosing and is used to create the legend.
* The next two lines add labels to the axis
* The last two lines show the legend and the plot

In [ ]:
fig,ax=plt.subplots(figsize=(20,4))
plt.plot(df.iloc[:,0], df.iloc[:,1], label = "Siderastrea sidera")
plt.plot(df.iloc[:,0], df.iloc[:,4], label = "Siderastrea sidera")
plt.plot(df.iloc[:,0], df.iloc[:,7], label = "Montastraea faveolata")
plt.xlabel('Time')
plt.ylabel('Sr/Ca (mmol/mol)')
plt.legend()
plt.show()

**Question:** What are the main takeaways from your plot? What does this mean for the calibration?

Now, let's examine the temperature record. Plot the two temperature series

In [ ]:
## Your code here

**Question:** How can you use the data for calibration?

One choice could be to only use the Sand Key data. Another would be to take the mean of the two series. Let's take the mean and create a new column in our dataframe.

In [ ]:
av_t = pd.DataFrame(data={'A':df.iloc[:,10],'B':df.iloc[:,12]}).mean(axis=1)
df[('Monthly mean SST','Average Temperature')]=av_t
df.head()

And plot:

In [ ]:
fig,ax=plt.subplots(figsize=(20,4))
plt.plot(df.iloc[:,0], df.iloc[:,14])
plt.xlabel('Time')
plt.ylabel('SST')
plt.show()

## Ordinary least square regression

### *Siderastrea sidera*

Let's start with the calibration for *Siderastrea sidera*. Let's plot the Sr/Ca vs the temperature for the two records.

In [ ]:
fig,ax=plt.subplots(figsize=(10,10))
plt.scatter(df.iloc[:,14], df.iloc[:,1],label='Coral 1')
plt.scatter(df.iloc[:,14], df.iloc[:,4],label='Coral 2')
plt.xlabel('SST')
plt.ylabel('Sr/Ca (mmol/mol)')
plt.legend()
plt.show()

**Question:** How do you interpret these results? Should we average the data? 

Let's consider them as separate samples. The cell below concatenates the data and plots the results. 

In [ ]:
X = np.concatenate((np.array(df.iloc[:,14]),np.array(df.iloc[:,14])))
Y = np.concatenate((np.array(df.iloc[:,1]),np.array(df.iloc[:,4])))

#Remove NaNs
idx_X = np.argwhere(~np.isnan(X))
idx_Y = np.argwhere(~np.isnan(Y))

idx = (np.intersect1d(idx_X, idx_Y))
X_train = X[idx]
Y_train = Y[idx]

fig,ax=plt.subplots(figsize=(10,10))
plt.scatter(X_train, Y_train)
plt.xlabel('SST')
plt.ylabel('Sr/Ca (mmol/mol)')
plt.show()

Now we're ready for the regression.

In [ ]:
lr = LinearRegression()
lr.fit(X_train.reshape((-1, 1)),Y_train)

'Sr/Ca = %0.2f' % lr.intercept_ + '%0.4f' %lr.coef_ + 'SST'

**Question:** How do your results compare to those of DeLong et al.? 

**Exercise:** Repeat the calibration with only one coral head, then the others. Are the results robust? Why would you want to check for this?

Python hint: To take only one column in consideration, you can use the following lines at the beginning:



```
X = np.array(df.iloc[:,14])
Y = np.array(df.iloc[:,1])

```
There is no need to concatenate. 


In [ ]:
## your code here

### *Montastraea faveolata*

Repeat the process for *Montastraea faveolata*.

## Bayesian Calibration

### *Siderastrea sidera*



Optional: If you are proficient with Python, feel free to play with different priors in the [PyMC3 package](https://pymc3-testing.readthedocs.io/en/rtd-docs/index.html). 

The code below does the following:

1. It assigns a model called `basic_model`
2. It defines the priors for the regression:
  * a is the slope. I chose a uniform prior over the -0.1 to 0 range. From previous studies, I know that the value of the slope is negative. Here I chose a small range for illustrative purposes. But if you have no idea what it should be, then you need to increase that range. On the other hand, if you have prior information about the value, then you can choose a different prior (e.g. normal distribution). **Warning:** Remember that the prior encodes your knowledge of the problem **BEFORE** you have seen the data. It is **not** appropriate to do an ordinary least square and use this as the prior.
  * b is the intercept. Here I chose a uniform distribution over the [5,20] interval
  * sigma represents the uncertainty in the posterior of the observations. It is fair to assume a Gaussian centered around the mean of the data as a prior. The trick is to choose the right sigma as to represent prior belief on how accurate and precise the measurements are. 
3. It defines the expected value of the outcome
4. It then creates a model for the posterior (Gaussian model seems appropriate)
5. Finally, it samples the space using the NUTS sampler. 

Bayesian regression are often solved numerically. Each sampler does this different but the idea is that the algorithm explores the space and calculates the cost of jumping from one value to another. When getting closer to the answers, the jumps are less so and convergence is achieved. 

Here, I asked to draw 5000 samples (5000 jumps) on two chains (each chain is initialized at different values, ensuring that the answer doesn't depend on the initial value), burning the first 500 samples (assuming that the fist guesses are very wrong) and retaining the last 4500 to form the distributions. 

In [ ]:
with pm.Model() as basic_model:
    
    # Prior is a uniform distribution over the -1 to 0 interval for the slope and 0 to 50 for intercept
    a = pm.Uniform('a',lower=-0.1, upper=0)
    b = pm.Uniform('b',lower=5, upper=20)
    sigma = pm.Normal('sigma', mu=8.9, sd=1)
    
    # Expected value of the outcome
    mu = b + a*X_train
    
    # Create the model
    Y_obs = pm.Normal('Y_obs', mu=mu, sd=sigma, observed=Y_train)

    #Perform NUTS sampling
    normal_trace = pm.sample(draws=5000, chains = 2, tune = 500)

Let's plot our results. 

1. The plots on the left represent the posterior distributions for sigma, a, and b. 

**Question:** Are these values consistent with the OLS performed above? 

2. The plots on the right represent the values that were tried during the sampling. You want to make sure that the values are hovering around (no long trend that would indicate no convergence). 

In [ ]:
_ = pm.traceplot(normal_trace)

You can also ask for a numerical summary:

In [ ]:
pm.summary(normal_trace)

**Exercise:** Repeat the calibration for *Montastraea faveolata*. 